<a href="https://colab.research.google.com/github/Ignowsky/Payroll-PDF-Parser/blob/main/Leitor_FOPAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 99.0 MB/s eta 0:00:00


# Versão 1.0

In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """Cria um nome de coluna limpo e padronizado, com regra especial para empréstimos."""
    descricao_upper = descricao.upper()
    if "EMP. CRED. TRAB" in descricao_upper or "EMPRESTIMO" in descricao_upper:
        return "9750_EMPRESTIMO_CONSIGNADO"

    descricao = re.sub(r'^\d+\s*', '', descricao)
    descricao_limpa = re.sub(r'[^a-zA-Z0-9\s]', '', descricao).strip()
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa)
    return f"{codigo}_{descricao_limpa}"

def extrair_info_base(texto_pagina):
    """Extrai a competência do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    return {'competencia': competencia_match.group(1) if competencia_match else 'N/A'}

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])

                info_base = extrair_info_base(texto_completo_pdf)

                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())

                blocos_encontrados = re.finditer(r'(?:Empr\.|Contr\.)\s*:\s*\d+.*?(?=\n(?:Empr\.|Contr\.)\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(0)

                    if not ("Situação:" in bloco and "CPF:" in bloco):
                        continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")

                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    # --- LÓGICA DE EXTRAÇÃO DO CABEÇALHO 100% CORRIGIDA ---
                    header_match = re.search(r'(?:Empr\.|Contr\.)\s*:\s*\d+\s+(.*?)\s+Situação:.*?CPF:\s*([\d\.\-]+)\s+Adm:\s*(\d{2}/\d{2}/\d{4})', bloco, re.DOTALL)
                    if header_match:
                        dados_funcionario['nome_funcionario'] = header_match.group(1).replace('\n', ' ').strip()
                        dados_funcionario['cpf'] = header_match.group(2)
                        dados_funcionario['data_admissao'] = header_match.group(3)

                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)\s+(?:C\.|С\.)', bloco, re.DOTALL)
                    if cargo_match:
                        dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip()

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match:
                        dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]
                        rubrica_pattern = re.compile(r'(\d+)\s+([A-ZÀ-Ú\d\s\.\/Nº\-\(\)]+?)\s+[\d\.,\s]+?([\d\.,]+)\s+[PD]')

                        for linha in tabela_str:
                            for match in rubrica_pattern.finditer(linha):
                                codigo, desc, valor = match.groups()
                                nome_col = limpar_nome_coluna(codigo.strip(), desc.strip())
                                valor_limpo = limpar_valor(valor)
                                dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    # Organiza as colunas de saída conforme solicitado
    colunas_info_pessoal = [
        'competencia', 'departamento', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/Teste'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOLHA_FINAL.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 1 PDFs para processar...

---> Processando arquivo: 10-Extrato Decimo Terceiro- 10-2025.pdf
    - Sucesso! Foram processados 61 funcionários neste arquivo.


--- Processo Finalizado com Sucesso! ---
Sua base de dados final foi salva no arquivo: /content/BASE_FOLHA_FINAL.csv


# Versão 1.1

In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """Cria um nome de coluna limpo e padronizado, com regra especial para empréstimos."""
    descricao_upper = descricao.upper()
    if "EMP. CRED. TRAB" in descricao_upper or "EMPRESTIMO" in descricao_upper:
        return "9750_EMPRESTIMO_CONSIGNADO"

    descricao = re.sub(r'^\d+\s*', '', descricao)
    descricao_limpa = re.sub(r'[^a-zA-Z0-9\s]', '', descricao).strip()
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa)
    return f"{codigo}_{descricao_limpa}"

def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        'competencia': competencia_match.group(1).strip() if competencia_match else 'N/A',
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else 'N/A'
    }

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])

                info_base = extrair_info_base(texto_completo_pdf)

                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())

                blocos_encontrados = re.finditer(r'(?:Empr\.|Contr\.)\s*:\s*\d+.*?(?=\n(?:Empr\.|Contr\.)\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(0)

                    if not ("Situação:" in bloco and "CPF:" in bloco):
                        continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")

                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    # --- LÓGICA DE EXTRAÇÃO DO CABEÇALHO CORRIGIDA PARA SER FLEXÍVEL ---
                    header_match = re.search(
                        r'(?:Empr\.|Contr\.)\s*:\s*\d+\s+(.*?)\s+Situação:.*?CPF:\s*([\d\.\-]+)(?:\s+Adm:\s*(\d{2}/\d{2}/\d{4}))?',
                        bloco,
                        re.DOTALL
                    )
                    if header_match:
                        dados_funcionario['nome_funcionario'] = header_match.group(1).replace('\n', ' ').strip()
                        dados_funcionario['cpf'] = header_match.group(2)
                        # A data de admissão agora é opcional
                        dados_funcionario['data_admissao'] = header_match.group(3) if header_match.group(3) else 'N/A'

                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)\s+(?:C\.|С\.)', bloco, re.DOTALL)
                    if cargo_match:
                        dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip()

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match:
                        dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]
                        # Regex mais robusto para capturar as verbas
                        rubrica_pattern = re.compile(r'(\d+)\s+(.*?)\s+([\d\.,]+)\s+[PD]')

                        for linha in tabela_str:
                            for match in rubrica_pattern.finditer(linha):
                                codigo, desc, valor = match.groups()
                                # Ignora a coluna de referência que pode ser confundida com valor
                                if not re.search(r'[a-zA-Z]', valor):
                                    nome_col = limpar_nome_coluna(codigo.strip(), desc.strip())
                                    valor_limpo = limpar_valor(valor)
                                    dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/Teste'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOLHA_COMPLETA_E_FUNCIONAL.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final e 100% funcional foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 2 PDFs para processar...

---> Processando arquivo: 10--Extrato Geral- 09-2025.pdf
    - Sucesso! Foram processados 69 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Decimo Terceiro- 10-2025.pdf
    - Sucesso! Foram processados 61 funcionários neste arquivo.


--- Processo Finalizado com Sucesso! ---
Sua base de dados final e 100% funcional foi salva no arquivo: /content/BASE_FOLHA_COMPLETA_E_FUNCIONAL.csv


# Versão 1.2

In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """Cria um nome de coluna limpo e padronizado, com regra especial para empréstimos."""
    descricao_upper = descricao.upper()
    if "EMP. CRED. TRAB" in descricao_upper or "EMPRESTIMO" in descricao_upper:
        return "9750_EMPRESTIMO_CONSIGNADO"

    descricao = re.sub(r'^\d+\s*', '', descricao)
    descricao_limpa = re.sub(r'[^a-zA-Z0-9\s]', '', descricao).strip()
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa)
    return f"{codigo}_{descricao_limpa}"

def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        'competencia': competencia_match.group(1).strip() if competencia_match else 'N/A',
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else 'N/A'
    }

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])

                info_base = extrair_info_base(texto_completo_pdf)

                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())

                # --- LÓGICA DE DIVISÃO DE BLOCOS 100% CORRIGIDA (PONTO OPCIONAL) ---
                blocos_encontrados = re.finditer(
                    r'((?:Empr|Contr)\.?\s*:\s*\d+.*?)(?=\n(?:Empr|Contr)\.?\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)',
                    texto_completo_pdf,
                    re.DOTALL
                )

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(1)

                    if not ("Situação:" in bloco and "CPF:" in bloco):
                        continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")

                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    # --- REGEX UNIFICADA E À PROVA DE FALHAS PARA O CABEÇALHO (PONTO OPCIONAL) ---
                    header_match = re.search(
                        r'(Empr|Contr)\.?\s*:\s*\d+\s+(.*?)\s+Situação:.*?CPF:\s*([\d\.\-]+)(?:\s+Adm:\s*(\d{2}/\d{2}/\d{4}))?',
                        bloco,
                        re.DOTALL
                    )

                    if header_match:
                        vinculo_raw, nome, cpf, admissao = header_match.groups()
                        dados_funcionario['vinculo'] = 'Empregado' if 'Empr' in vinculo_raw else 'Contribuinte'
                        dados_funcionario['nome_funcionario'] = nome.replace('\n', ' ').strip()
                        dados_funcionario['cpf'] = cpf
                        dados_funcionario['data_admissao'] = admissao if admissao else 'N/A'

                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)\s+(?:C\.|С\.)', bloco, re.DOTALL)
                    if cargo_match:
                        dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip()

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match:
                        dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]
                        rubrica_pattern = re.compile(r'(\d+)\s+(.*?)\s+([\d\.,]+)\s+[PD]')

                        for linha in tabela_str:
                            for match in rubrica_pattern.finditer(linha):
                                codigo, desc, valor = match.groups()
                                if not re.search(r'[a-zA-Z]', valor):
                                    nome_col = limpar_nome_coluna(codigo.strip(), desc.strip())
                                    valor_limpo = limpar_valor(valor)
                                    dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/FOPAG'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOLHA_DEFINITIVA_FINAL.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 37 PDFs para processar...

---> Processando arquivo: 01.2023 ARQDIGITAL - Folha de Pagamento c.Prolabore Carol.pdf
    - Sucesso! Foram processados 39 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Folha- 08-2023.pdf
    - Sucesso! Foram processados 47 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Mensal-03-2024.pdf
    - Sucesso! Foram processados 66 funcionários neste arquivo.

---> Processando arquivo: ARQ - 1ª Parcela 13.2023.pdf
    - Sucesso! Foram processados 42 funcionários neste arquivo.

---> Processando arquivo: 05.2023 ARQDIGITAL - Folha de Pagamento.pdf
    - Sucesso! Foram processados 40 funcionários neste arquivo.

---> Processando arquivo: 04.2023 ARQDIGITAL - Folha de Pagamento.pdf
    - Sucesso! Foram processados 40 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Folha- 01-2024.pdf
    - Sucesso! Foram processados 62 funcionários neste arquivo.

---> Processando arquivo: 12.2024 ARQDIGITAL - Folha de

# Versão 2


In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """
    Cria um nome de coluna limpo e padronizado, consolidando verbas
    pelo CÓDIGO como regra principal, com base em um mapa abrangente.
    """
    mapeamento_codigos = {
        '12': 'P_12_13_Salario_Integral',
        '13': 'P_13_13_Salario_Adiantamento',
        '19': 'P_19_Retroativo_Salarial',
        '22': 'P_22_Aviso_Previo',
        '28': 'P_28_Ferias_Vencidas',
        '29': 'P_29_Ferias_Proporcionais',
        '48': 'D_48_Vale_Transporte',
        '49': 'P_49_Aviso_Previo_Nao_Trabalhado',
        '50': 'P_50_Adiantamento_13_Salario',
        '51': 'D_51_Liquido_Rescisao',
        '64': 'P_64_1_3_Ferias_Rescisao',
        '150': 'P_150_Horas_Extras_50',
        '200': 'P_200_Horas_Extras_100',
        '241': 'D_241_Desc_Vale_Transporte',
        '242': 'P_242_Honorarios',
        '246': 'P_246_Diferenca_Salarial',
        '250': 'P_250_Reflexo_Extra_DSR',
        '258': 'P_258_Anuenio_Sindpd_PA',
        '263': 'P_263_Pag_Banco_Horas',
        '276': 'P_276_Trienio_Sindpd',
        '283': 'P_283_VT_Mes_Seguinte',
        '286': 'D_286_Desc_Plano_Medico_Dep',
        '291': 'D_291_Desc_Banco_Horas',
        '295': 'P_295_Hora_Extra_50',
        '296': 'D_296_VT_Nao_Utilizado',
        '297': 'D_297_VA_Nao_Utilizado',
        '311': 'D_311_Desc_2_Via_Cartao',
        '314': 'P_314_Dev_Desc_Indevido',
        '316': 'P_316_Devolucao_Desc_Plano_Odonto',
        '317': 'P_317_Dev_Desc_Plano_Odonto',
        '325': 'D_325_Desc_Plano_Odonto',
        '331': 'D_331_Desc_Banco_Horas',
        '340': 'P_340_Adicional_Noturno',
        '362': 'D_362_Desconto_VA_VR',
        '375': 'D_375_Desconto_Plano_Saude_Dep_F',
        '379': 'D_379_Desconto_Plano_Odonto_F',
        '394': 'D_394_Desconto_Diversos',
        '399': 'P_399_Banco_Horas_Pago',
        '447': 'D_447_Desc_Plano_Odonto_Alfa_Dep',
        '449': 'D_449_Desc_Plano_Odonto_Beta',
        '451': 'D_451_Desc_Plano_Odonto_Alfa_Dep_F',
        '453': 'D_453_Desc_Plano_Odonto_Beta_F',
        '461': 'P_461_Gratificacao_Funcao',
        '572': 'P_572_Dev_Desc_Plano_Odonto',
        '574': 'P_574_Gratificacao',
        '623': 'P_623_Gratificacao_Funcao',
        '637': 'D_637_Taxa_Campanha_Sindical',
        '639': 'D_639_Desconto_Valor_Pago',
        '695': 'P_695_Bolsa_Auxilio_Bonificacao',
        '700': 'P_700_Dev_Desc_INSS_Maior',
        '725': 'P_725_Dif_Plano_Medico_Dep',
        '763': 'P_763_Reembolso_Conselho',
        '777': 'D_777_VT_VA_Nao_Utilizado',
        '800': 'P_800_Media_Horas_13',
        '801': 'P_801_Media_Valor_13',
        '802': 'P_802_Media_Fixa_13',
        '803': 'P_803_13_1_12_Indenizado',
        '804': 'D_804_IRRF_13',
        '805': 'P_805_Media_Valor_Ferias',
        '806': 'P_806_Media_Horas_Ferias',
        '807': 'P_807_Media_Fixa_Ferias',
        '808': 'P_808_Media_Valor_Abono',
        '809': 'P_809_Media_Horas_Abono',
        '810': 'P_810_Media_Fixa_Abono',
        '811': 'P_811_Ferias_1_12_Indenizado',
        '812': 'D_812_INSS_Ferias',
        '937': "D_937_Adiantamento_Ferias",
        '1015': 'P_1015_Anuenio_Sindpd_PA',
        '8069': 'D_8069_Faltas_Horas_Parciais',
        '8104': 'P_8104_13_Salario_Maternidade',
        '8111': 'D_8111_Desc_Plano_Saude_Dep',
        '8112': 'P_8112_Dif_13_Ferias',
        '8126': 'P_8126_1_3_Ferias_Indenizada_Resc',
        '8128': 'D_8128_IRRF_Dif_Ferias',
        '8130': 'P_8130_Estouro_Rescisao',
        '8158': 'P_8158_Media_Ferias_1_12_Indenizado',
        '8169': 'P_8169_1_3_Ferias_Proporcionais_Resc',
        '9750': 'D_9750_Desc_Emprestimo_Consignado',
        '1069': 'D_1069_Desc_Emprestimo_Consignado',
        '766': 'P_766_Dif_Trienio',
        '817': 'P_817_Media_Fer_Proporcionais',
        '8181': 'P_8181_Dif_Media_Hora_13',
        '8182': 'P_8182_Dif_Media_Valor_13',
        '8184': 'P_8184_Dif_Adicional_13',
        '8189': 'P_8189_Dif_Media_Horas_Ferias',
        '8190': 'P_8190_Dif_Media_Valor_Ferias',
        '8192': 'P_8192_Dif_Media_Valor_Ferias',
        '8197': 'P_8197_Dif_Media_Horas_Abono_Ferias',
        '8200': 'P_8200_Dif_Adicional_Abono_Ferias',
        '820': 'P_820_Media_Ferias_Vencidas',
        '821': 'D_821_Dif_Inss_Ferias',
        '825': 'D_825_Inss_13_Salario',
        '826': 'D_826_Inss_Sobre_Rescisao',
        '828': 'D_828_Irrf_Rescisao',
        '833': 'P_833_Media_Horas_13_Adiantado',
        '834': 'P_834_Media_Valor_13_Adiantado',
        '835': 'P_835_Adiocional_Fixo_13_Adiantado',
        '836': 'P_836_Ajuste_Inss',
        '8392': 'P_8392_13_Salario_Adiantado_Ferias',
        '8393': 'P_8393_Media_Horas_13_Adiantado_Ferias',
        '8394': 'P_8394_Media_Valor_13_Adiantado_Ferias',
        '8396': 'P_8396_Vantagem_13_Adiantado',
        '8417': 'P_8417_Dif_1_3_Abono_Ferias',
        '846': 'P_846_Dif_Abono_Ferias',
        '842': 'D_842_Multa_Estabilidade_Art_482',
        '843': 'D_843_Inss_Empregador',
        '8490': 'P_8490_Bolsa_Auxilio_Ferias_Proporcionais',
        '854': 'P_854_Reflexo_Adicional_Noturno_DSR',
        '8781': 'P_8781_Salario_Empregado',
        '8550': 'P_8550_13_Salario_Integral_Rescisao',
        '8553': 'P_8553_Media_13_Rescisao',
        '856': 'D_856_Irrf_Empregador',
        '869': 'D_869_ISS',
        '8783': 'P_8783_Dias_Ferias',
        '8800': 'P_8800_Dias_Abono(Ferias)',
        '931': 'P_931_1_3_Ferias',
        '932': 'P_932_1_3_Abono_Ferias',
        '940': 'P_940_Diferenca_Ferias',
        '8784': 'P_8784_Salario_Maternidade_Dias',
        '998': 'D_998_INSS',
        '999': 'D_999_IRRF',
        '8791': 'P_8791_Dias_Afast_Dir_Integrais',
        '8797': 'P_8797_Dias_Bolsa_Estagio',
        '8832': 'P_8832_Dias_Licença_Maternidade',
        '8870': 'P_8870_Dias_Afast_Doenca_Dir_Integrais',
        '919': 'P_919_Trienio_Sinpd',
        '964': 'D_964_Desc_Odonto_Mais_Clarear',
        '8918': 'D_8918_Adiantamento_13_Media_Valor',
        '8921': 'D_8921_Adiantamento_13_Media_Fixa',
        '8919': 'D_8919_Adiantamento_13_Media_Horas',
        '9180': 'P_9180_Saldo_Salario_Dias',
        '9591': 'P_9591_Aviso_Previo',
        '9592': 'P_9592_13_1_12_Indenizado',
        '9598': 'P_9598_Vantagem_Aviso_Indenizado',
        '9602': 'P_9602_Vantagem_13_1_12_Indenizado',
        '9380': 'P_9380_Pro_Labore_Dias',
        '942': 'D_942_Irrf_Ferias',
        '963': 'D_963_Desc_Odonto_Mais_Orto',
        '965': 'D_963_Desc_Odonto_Mais_Doc',
        '989': 'D_989_Inss_13_Sal_Rescisao',
        '995': 'P_995_Salario_Familia'

    }

    if codigo in mapeamento_codigos:
        return mapeamento_codigos[codigo]

    descricao_limpa = re.sub(r'[^a-zA-Z\s]', '', descricao).strip()
    primeira_palavra = descricao_limpa.split(' ')[0] if descricao_limpa else ''
    # Ensure a string is always returned
    return f"{codigo}_{primeira_palavra.upper()}_TOTAL" if primeira_palavra else f"{codigo}_DESCRICAO_NAO_IDENTIFICADA"


def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        'competencia': competencia_match.group(1).strip() if competencia_match else 'N/A',
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else 'N/A'
    }

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])
                info_base = extrair_info_base(texto_completo_pdf)
                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())
                blocos_encontrados = re.finditer(r'((?:Empr|Contr)\.?\s*:\s*\d+.*?)(?=\n(?:Empr|Contr)\.?\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(1)
                    if not ("Situação:" in bloco and "CPF:" in bloco): continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")
                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    header_match = re.search(r'(Empr|Contr)\.?\s*:\s*\d+\s+(.*?)\s+Situação:.*?CPF:\s*([\d\.\-]+)(?:\s+Adm:\s*(\d{2}/\d{2}/\d{4}))?', bloco, re.DOTALL)
                    if header_match:
                        vinculo_raw, nome, cpf, admissao = header_match.groups()
                        dados_funcionario.update({
                            'vinculo': 'Empregado' if 'Empr' in vinculo_raw else 'Contribuinte',
                            'nome_funcionario': nome.replace('\n', ' ').strip(), 'cpf': cpf,
                            'data_admissao': admissao if admissao else 'N/A'
                        })

                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)\s+(?:C\.|С\.)', bloco, re.DOTALL)
                    if cargo_match: dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip()

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match: dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]

                        for linha in tabela_str:
                            # --- LÓGICA DE EXTRAÇÃO DE VERBAS 100% CORRIGIDA ---
                            match_codigo = re.match(r'^\s*(\d+)', linha)
                            match_valor = re.search(r'([\d\.,]+)\s+([PD])\s*$', linha)

                            if match_codigo and match_valor:
                                codigo = match_codigo.group(1)
                                valor = match_valor.group(1)

                                # Isola o "miolo" entre o código e o valor
                                start_index = match_codigo.end()
                                end_index = match_valor.start()
                                miolo = linha[start_index:end_index].strip()

                                # Remove o valor de referência do final do miolo para obter a descrição limpa
                                desc_limpa = re.sub(r'\s*[\d\.,%]+$', '', miolo).strip()

                                nome_col = limpar_nome_coluna(codigo, desc_limpa)
                                valor_limpo = limpar_valor(valor)
                                dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/Teste'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOPAAG_STAGGIN.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 2 PDFs para processar...

---> Processando arquivo: 10--Extrato Geral- 09-2025.pdf
    - Sucesso! Foram processados 71 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Decimo Terceiro- 10-2025.pdf
    - Sucesso! Foram processados 61 funcionários neste arquivo.


--- Processo Finalizado com Sucesso! ---
Sua base de dados final foi salva no arquivo: /content/BASE_FOPAAG_STAGGIN.csv


# Versão 2.1

In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """
    Cria um nome de coluna limpo e padronizado, consolidando verbas
    pelo CÓDIGO como regra principal, com base em um mapa abrangente.
    """
    # DICIONÁRIO ATUALIZADO COM AS NOVAS VERBAS
    mapeamento_codigos = {
        '12': 'P_12_13_Salario_Integral',
        '13': 'P_13_13_Salario_Adiantamento',
        '19': 'P_19_Retroativo_Salarial',
        '22': 'P_22_Aviso_Previo',
        '28': 'P_28_Ferias_Vencidas',
        '29': 'P_29_Ferias_Proporcionais',
        '48': 'D_48_Vale_Transporte',
        '49': 'P_49_Aviso_Previo_Nao_Trabalhado',
        '50': 'P_50_Adiantamento_13_Salario',
        '51': 'D_51_Liquido_Rescisao',
        '64': 'P_64_1_3_Ferias_Rescisao',
        '150': 'P_150_Horas_Extras_50',
        '200': 'P_200_Horas_Extras_100',
        '241': 'D_241_Desc_Vale_Transporte',
        '242': 'P_242_Honorarios',
        '246': 'P_246_Diferenca_Salarial',
        '250': 'P_250_Reflexo_Extra_DSR',
        '258': 'P_258_Anuenio_Sindpd_PA',
        '263': 'P_263_Pag_Banco_Horas',
        '276': 'P_276_Trienio_Sindpd',
        '283': 'P_283_VT_Mes_Seguinte',
        '286': 'D_286_Desc_Plano_Medico_Dep',
        '291': 'D_291_Desc_Banco_Horas',
        '295': 'P_295_Hora_Extra_50',
        '296': 'D_296_VT_Nao_Utilizado',
        '297': 'D_297_VA_Nao_Utilizado',
        '311': 'D_311_Desc_2_Via_Cartao',
        '314': 'P_314_Dev_Desc_Indevido',
        '316': 'P_316_Devolucao_Desc_Plano_Odonto',
        '317': 'P_317_Dev_Desc_Plano_Odonto',
        '325': 'D_325_Desc_Plano_Odonto',
        '331': 'D_331_Desc_Banco_Horas',
        '340': 'P_340_Adicional_Noturno',
        '362': 'D_362_Desconto_VA_VR',
        '375': 'D_375_Desconto_Plano_Saude_Dep_F',
        '379': 'D_379_Desconto_Plano_Odonto_F',
        '394': 'D_394_Desconto_Diversos',
        '399': 'P_399_Banco_Horas_Pago',
        '447': 'D_447_Desc_Plano_Odonto_Alfa_Dep',
        '449': 'D_449_Desc_Plano_Odonto_Beta',
        '451': 'D_451_Desc_Plano_Odonto_Alfa_Dep_F',
        '453': 'D_453_Desc_Plano_Odonto_Beta_F',
        '461': 'P_461_Gratificacao_Funcao',
        '572': 'P_572_Dev_Desc_Plano_Odonto',
        '574': 'P_574_Gratificacao',
        '623': 'P_623_Gratificacao_Funcao',
        '637': 'D_637_Taxa_Campanha_Sindical',
        '639': 'D_639_Desconto_Valor_Pago',
        '695': 'P_695_Bolsa_Auxilio_Bonificacao',
        '700': 'P_700_Dev_Desc_INSS_Maior',
        '725': 'P_725_Dif_Plano_Medico_Dep',
        '763': 'P_763_Reembolso_Conselho',
        '777': 'D_777_VT_VA_Nao_Utilizado',
        '800': 'P_800_Media_Horas_13',
        '801': 'P_801_Media_Valor_13',
        '802': 'P_802_Media_Fixa_13',
        '803': 'P_803_13_1_12_Indenizado',
        '804': 'D_804_IRRF_13',
        '805': 'P_805_Media_Valor_Ferias',
        '806': 'P_806_Media_Horas_Ferias',
        '807': 'P_807_Media_Fixa_Ferias',
        '808': 'P_808_Media_Valor_Abono',
        '809': 'P_809_Media_Horas_Abono',
        '810': 'P_810_Media_Fixa_Abono',
        '811': 'P_811_Ferias_1_12_Indenizado',
        '812': 'D_812_INSS_Ferias',
        '937': "D_937_Adiantamento_Ferias",
        '1015': 'P_1015_Anuenio_Sindpd_PA',
        '8069': 'D_8069_Faltas_Horas_Parciais',
        '8104': 'P_8104_13_Salario_Maternidade',
        '8111': 'D_8111_Desc_Plano_Saude_Dep',
        '8112': 'P_8112_Dif_13_Ferias',
        '8126': 'P_8126_1_3_Ferias_Indenizada_Resc',
        '8128': 'D_8128_IRRF_Dif_Ferias',
        '8130': 'P_8130_Estouro_Rescisao',
        '8158': 'P_8158_Media_Ferias_1_12_Indenizado',
        '8169': 'P_8169_1_3_Ferias_Proporcionais_Resc',
        '9750': 'D_9750_Desc_Emprestimo_Consignado',
        '1069': 'D_1069_Desc_Emprestimo_Consignado',
        '766': 'P_766_Dif_Trienio',
        '817': 'P_817_Media_Fer_Proporcionais',
        '8181': 'P_8181_Dif_Media_Hora_13',
        '8182': 'P_8182_Dif_Media_Valor_13',
        '8184': 'P_8184_Dif_Adicional_13',
        '8189': 'P_8189_Dif_Media_Horas_Ferias',
        '8190': 'P_8190_Dif_Media_Valor_Ferias',
        '8192': 'P_8192_Dif_Media_Valor_Ferias',
        '8197': 'P_8197_Dif_Media_Horas_Abono_Ferias',
        '8200': 'P_8200_Dif_Adicional_Abono_Ferias',
        '820': 'P_820_Media_Ferias_Vencidas',
        '821': 'D_821_Dif_Inss_Ferias',
        '825': 'D_825_Inss_13_Salario',
        '826': 'D_826_Inss_Sobre_Rescisao',
        '828': 'D_828_Irrf_Rescisao',
        '833': 'P_833_Media_Horas_13_Adiantado',
        '834': 'P_834_Media_Valor_13_Adiantado',
        '835': 'P_835_Adiocional_Fixo_13_Adiantado',
        '836': 'P_836_Ajuste_Inss',
        '8392': 'P_8392_13_Salario_Adiantado_Ferias',
        '8393': 'P_8393_Media_Horas_13_Adiantado_Ferias',
        '8394': 'P_8394_Media_Valor_13_Adiantado_Ferias',
        '8396': 'P_8396_Vantagem_13_Adiantado',
        '8417': 'P_8417_Dif_1_3_Abono_Ferias',
        '846': 'P_846_Dif_Abono_Ferias',
        '842': 'D_842_Multa_Estabilidade_Art_482',
        '843': 'D_843_Inss_Empregador',
        '8490': 'P_8490_Bolsa_Auxilio_Ferias_Proporcionais',
        '854': 'P_854_Reflexo_Adicional_Noturno_DSR',
        '8781': 'P_8781_Salario_Empregado',
        '8550': 'P_8550_13_Salario_Integral_Rescisao',
        '8553': 'P_8553_Media_13_Rescisao',
        '856': 'D_856_Irrf_Empregador',
        '869': 'D_869_ISS',
        '8783': 'P_8783_Dias_Ferias',
        '8800': 'P_8800_Dias_Abono(Ferias)',
        '931': 'P_931_1_3_Ferias',
        '932': 'P_932_1_3_Abono_Ferias',
        '940': 'P_940_Diferenca_Ferias',
        '8784': 'P_8784_Salario_Maternidade_Dias',
        '998': 'D_998_INSS',
        '999': 'D_999_IRRF',
        '8791': 'P_8791_Dias_Afast_Dir_Integrais',
        '8797': 'P_8797_Dias_Bolsa_Estagio',
        '8832': 'P_8832_Dias_Licença_Maternidade',
        '8870': 'P_8870_Dias_Afast_Doenca_Dir_Integrais',
        '919': 'P_919_Trienio_Sinpd',
        '964': 'D_964_Desc_Odonto_Mais_Clarear',
        '8918': 'D_8918_Adiantamento_13_Media_Valor',
        '8921': 'D_8921_Adiantamento_13_Media_Fixa',
        '8919': 'D_8919_Adiantamento_13_Media_Horas',
        '9180': 'P_9180_Saldo_Salario_Dias',
        '9591': 'P_9591_Aviso_Previo',
        '9592': 'P_9592_13_1_12_Indenizado',
        '9598': 'P_9598_Vantagem_Aviso_Indenizado',
        '9602': 'P_9602_Vantagem_13_1_12_Indenizado',
        '9380': 'P_9380_Pro_Labore_Dias',
        '942': 'D_942_Irrf_Ferias',
        '963': 'D_963_Desc_Odonto_Mais_Orto',
        '965': 'D_963_Desc_Odonto_Mais_Doc',
        '989': 'D_989_Inss_13_Sal_Rescisao',
        '995': 'P_995_Salario_Familia',
        '858': 'D_858_INSS_Autonomo',
        '827': 'D_827_IRRF_13_Salario_Rescisao'
    }

    codigo_limpo = str(codigo).strip()
    if codigo_limpo in mapeamento_codigos:
        return mapeamento_codigos[codigo_limpo]

    # Fallback para casos não mapeados
    descricao_limpa = re.sub(r'[\d\s/]+$', '', descricao).strip() # Remove números e barras no final
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa)
    return f"NAO_MAPEADO_{codigo_limpo}_{descricao_limpa.upper()}"


def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        'competencia': competencia_match.group(1).strip() if competencia_match else 'N/A',
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else 'N/A'
    }

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])
                info_base = extrair_info_base(texto_completo_pdf)
                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())
                blocos_encontrados = re.finditer(r'((?:Empr|Contr)\.?\s*:\s*\d+.*?)(?=\n(?:Empr|Contr)\.?\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(1)
                    if not ("Situação:" in bloco and "CPF:" in bloco): continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")
                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    header_match = re.search(r'(Empr|Contr)\.?\s*:\s*\d+\s+(.*?)\s+Situação:.*?CPF:\s*([\d\.\-]+)(?:\s+Adm:\s*(\d{2}/\d{2}/\d{4}))?', bloco, re.DOTALL)
                    if header_match:
                        vinculo_raw, nome, cpf, admissao = header_match.groups()
                        dados_funcionario.update({
                            'vinculo': 'Empregado' if 'Empr' in vinculo_raw else 'Contribuinte',
                            'nome_funcionario': nome.replace('\n', ' ').strip(), 'cpf': cpf,
                            'data_admissao': admissao if admissao else 'N/A'
                        })

                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)\s+(?:C\.|С\.)', bloco, re.DOTALL)
                    if cargo_match: dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip()

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match: dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]

                        # --- INÍCIO DA LÓGICA DE EXTRAÇÃO CORRIGIDA ---
                        for linha in tabela_str:
                            if not re.search(r'\d', linha):
                                continue

                            # Regex mais flexível para capturar verbas.
                            # Captura: (código), (descrição com valor de referência opcional), (valor final), (tipo P/D)
                            # Funciona encontrando o padrão de um valor monetário + P/D no final da string/substring.
                            padrao_verba = r'(\d+)\s+(.*?)\s+([\d\.,]+)\s+([PD])\s*(?=\s+\d{2,}|$)'

                            for match in re.finditer(padrao_verba, linha):
                                codigo = match.group(1)
                                descricao_bruta = match.group(2).strip()
                                valor = match.group(3)

                                nome_col = limpar_nome_coluna(codigo, descricao_bruta)
                                valor_limpo = limpar_valor(valor)
                                dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo
                        # --- FIM DA LÓGICA DE EXTRAÇÃO ---

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/Teste'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOPAAG_STAGGIN.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 2 PDFs para processar...

---> Processando arquivo: 10--Extrato Geral- 09-2025.pdf
    - Sucesso! Foram processados 71 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Decimo Terceiro- 10-2025.pdf
    - Sucesso! Foram processados 61 funcionários neste arquivo.


--- Processo Finalizado com Sucesso! ---
Sua base de dados final foi salva no arquivo: /content/BASE_FOPAAG_STAGGIN.csv


# Versão 2.2 - Final


In [ ]:
#@title
import pdfplumber
import pandas as pd
import os
import re

def limpar_valor(valor_str):
    """Converte uma string monetária para float."""
    if isinstance(valor_str, str):
        return float(valor_str.replace('.', '').replace(',', '.'))
    return valor_str

def limpar_nome_coluna(codigo, descricao):
    """
    Cria um nome de coluna limpo e padronizado, consolidando verbas
    pelo CÓDIGO como regra principal, com base em um mapa abrangente.
    """
    mapeamento_original = {
        '12': 'P_12_13_Salario_Integral', '13': 'P_13_13_Salario_Adiantamento', '19': 'P_19_Retroativo_Salarial',
        '22': 'P_22_Aviso_Previo', '28': 'P_28_Ferias_Vencidas', '29': 'P_29_Ferias_Proporcionais',
        '49': 'P_49_Aviso_Previo_Nao_Trabalhado', '50': 'P_50_Adiantamento_13_Salario', '64': 'P_64_1_3_Ferias_Rescisao',
        '150': 'P_150_Horas_Extras_50', '200': 'P_200_Horas_Extras_100', '242': 'P_242_Honorarios',
        '246': 'P_246_Diferenca_Salarial', '250': 'P_250_Reflexo_Extra_DSR', '258': 'P_258_Anuenio_Sindpd_PA',
        '263': 'P_263_Pag_Banco_Horas', '276': 'P_276_Trienio_Sindpd', '283': 'P_283_VT_Mes_Seguinte',
        '295': 'P_295_Hora_Extra_50', '314': 'P_314_Dev_Desc_Indevido', '316': 'P_316_Devolucao_Desc_Plano_Odonto',
        '317': 'P_317_Dev_Desc_Plano_Odonto', '340': 'P_340_Adicional_Noturno', '399': 'P_399_Banco_Horas_Pago',
        '461': 'P_461_Gratificacao_Funcao', '572': 'P_572_Dev_Desc_Plano_Odonto', '574': 'P_574_Gratificacao',
        '623': 'P_623_Gratificacao_Funcao', '695': 'P_695_Bolsa_Auxilio_Bonificacao', '700': 'P_700_Dev_Desc_INSS_Maior',
        '725': 'P_725_Dif_Plano_Medico_Dep', '763': 'P_763_Reembolso_Conselho', '766': 'P_766_Dif_Trienio',
        '800': 'P_800_Media_Horas_13', '801': 'P_801_Media_Valor_13', '802': 'P_802_Media_Fixa_13',
        '803': 'P_803_13_1_12_Indenizado', '805': 'P_805_Media_Valor_Ferias', '806': 'P_806_Media_Horas_Ferias',
        '807': 'P_807_Media_Fixa_Ferias', '808': 'P_808_Media_Valor_Abono', '809': 'P_809_Media_Horas_Abono',
        '810': 'P_810_Media_Fixa_Abono', '811': 'P_811_Ferias_1_12_Indenizado', '817': 'P_817_Media_Fer_Proporcionais',
        '820': 'P_820_Media_Ferias_Vencidas', '833': 'P_833_Media_Horas_13_Adiantado', '834': 'P_834_Media_Valor_13_Adiantado',
        '835': 'P_835_Adiocional_Fixo_13_Adiantado', '836': 'P_836_Ajuste_Inss', '846': 'P_846_Dif_Abono_Ferias',
        '854': 'P_854_Reflexo_Adicional_Noturno_DSR', '919': 'P_919_Trienio_Sinpd', '931': 'P_931_1_3_Ferias',
        '932': 'P_932_1_3_Abono_Ferias', '940': 'P_940_Diferenca_Ferias', '995': 'P_995_Salario_Familia',
        '1015': 'P_1015_Anuenio_Sindpd_PA', '8104': 'P_8104_13_Salario_Maternidade', '8112': 'P_8112_Dif_13_Ferias',
        '8126': 'P_8126_1_3_Ferias_Indenizada_Resc', '8130': 'P_8130_Estouro_Rescisao', '8158': 'P_8158_Media_Ferias_1_12_Indenizado',
        '8169': 'P_8169_1_3_Ferias_Proporcionais_Resc', '8181': 'P_8181_Dif_Media_Hora_13', '8182': 'P_8182_Dif_Media_Valor_13',
        '8184': 'P_8184_Dif_Adicional_13', '8189': 'P_8189_Dif_Media_Horas_Ferias', '8190': 'P_8190_Dif_Media_Valor_Ferias',
        '8192': 'P_8192_Dif_Media_Valor_Ferias', '8197': 'P_8197_Dif_Media_Horas_Abono_Ferias', '8200': 'P_8200_Dif_Adicional_Abono_Ferias',
        '8392': 'P_8392_13_Salario_Adiantado_Ferias', '8393': 'P_8393_Media_Horas_13_Adiantado_Ferias', '8394': 'P_8394_Media_Valor_13_Adiantado_Ferias',
        '8396': 'P_8396_Vantagem_13_Adiantado', '8417': 'P_8417_Dif_1_3_Abono_Ferias', '8490': 'P_8490_Bolsa_Auxilio_Ferias_Proporcionais',
        '8550': 'P_8550_13_Salario_Integral_Rescisao', '8553': 'P_8553_Media_13_Rescisao', '8781': 'P_8781_Salario_Empregado',
        '8783': 'P_8783_Dias_Ferias', '8784': 'P_8784_Salario_Maternidade_Dias', '8791': 'P_8791_Dias_Afast_Dir_Integrais',
        '8797': 'P_8797_Dias_Bolsa_Estagio', '8800': 'P_8800_Dias_Abono(Ferias)', '8832': 'P_8832_Dias_Licença_Maternidade',
        '8870': 'P_8870_Dias_Afast_Doenca_Dir_Integrais', '9180': 'P_9180_Saldo_Salario_Dias', '9380': 'P_9380_Pro_Labore_Dias',
        '9591': 'P_9591_Aviso_Previo', '9592': 'P_9592_13_1_12_Indenizado', '9598': 'P_9598_Vantagem_Aviso_Indenizado',
        '9602': 'P_9602_Vantagem_13_1_12_Indenizado',
        '48': 'D_48_Vale_Transporte', '51': 'D_51_Liquido_Rescisao', '241': 'D_241_Desc_Vale_Transporte',
        '286': 'D_286_Desc_Plano_Medico_Dep', '291': 'D_291_Desc_Banco_Horas', '296': 'D_296_VT_Nao_Utilizado',
        '297': 'D_297_VA_Nao_Utilizado', '311': 'D_311_Desc_2_Via_Cartao', '325': 'D_325_Desc_Plano_Odonto',
        '331': 'D_331_Desc_Banco_Horas', '362': 'D_362_Desconto_VA_VR', '375': 'D_375_Desconto_Plano_Saude_Dep_F',
        '379': 'D_379_Desconto_Plano_Odonto_F', '394': 'D_394_Desconto_Diversos', '447': 'D_447_Desc_Plano_Odonto_Alfa_Dep',
        '449': 'D_449_Desc_Plano_Odonto_Beta', '451': 'D_451_Desc_Plano_Odonto_Alfa_Dep_F', '453': 'D_453_Desc_Plano_Odonto_Beta_F',
        '637': 'D_637_Taxa_Campanha_Sindical', '639': 'D_639_Desconto_Valor_Pago', '777': 'D_777_VT_VA_Nao_Utilizado',
        '804': 'D_804_IRRF_13', '812': 'D_812_INSS_Ferias', '821': 'D_821_Dif_Inss_Ferias',
        '825': 'D_825_Inss_13_Salario', '826': 'D_826_Inss_Sobre_Rescisao', '827': 'D_827_IRRF_13_Salario_Rescisao',
        '828': 'D_828_Irrf_Rescisao', '842': 'D_842_Multa_Estabilidade_Art_482', '843': 'D_843_Inss_Empregador',
        '856': 'D_856_Irrf_Empregador', '858': 'D_858_INSS_Autonomo', '869': 'D_869_ISS',
        '937': 'D_937_Adiantamento_Ferias', '942': 'D_942_Irrf_Ferias', '963': 'D_963_Desc_Odonto_Mais_Orto',
        '964': 'D_964_Desc_Odonto_Mais_Clarear', '965': 'D_963_Desc_Odonto_Mais_Doc', '989': 'D_989_Inss_13_Sal_Rescisao',
        '998': 'D_998_INSS', '999': 'D_999_IRRF', '1069': 'D_1069_Desc_Emprestimo_Consignado',
        '8069': 'D_8069_Faltas_Horas_Parciais', '8111': 'D_8111_Desc_Plano_Saude_Dep', '8128': 'D_8128_IRRF_Dif_Ferias',
        '8918': 'D_8918_Adiantamento_13_Media_Valor', '8919': 'D_8919_Adiantamento_13_Media_Horas', '8921': 'D_8921_Adiantamento_13_Media_Fixa',
        '9750': 'D_9750_Desc_Emprestimo_Consignado', '8214': 'D_8214_INSS_Dif_13_Salario', '8215': 'D_8215_IRRF_Dif_13_Salario',
        '8517': 'D_8517_Liquido_Rescisao_Estagiario', '8566': 'D_8566_Adiantamento_13_Salario_Rescisao'
    }

    proventos = {k: v for k, v in mapeamento_original.items() if v.startswith('P_')}
    descontos = {k: v for k, v in mapeamento_original.items() if v.startswith('D_')}
    sorted_proventos = dict(sorted(proventos.items(), key=lambda item: int(item[0])))
    sorted_descontos = dict(sorted(descontos.items(), key=lambda item: int(item[0])))
    mapeamento_codigos = {**sorted_proventos, **sorted_descontos}

    codigo_limpo = str(codigo).strip()
    if codigo_limpo in mapeamento_codigos:
        return mapeamento_codigos[codigo_limpo]

    descricao_limpa = re.sub(r'[\d\s/]+$', '', descricao).strip()
    descricao_limpa = re.sub(r'\s+', '_', descricao_limpa)
    return f"NAO_MAPEADO_{codigo_limpo}_{descricao_limpa.upper()}"


def extrair_info_base(texto_pagina):
    """Extrai a competência e o tipo de cálculo do documento."""
    competencia_match = re.search(r'Competência:\s*(\d{2}/\d{4})', texto_pagina)
    calculo_match = re.search(r'Cálculo\s*:\s*(.+)', texto_pagina)
    return {
        'competencia': competencia_match.group(1).strip() if competencia_match else 'N/A',
        'tipo_calculo': calculo_match.group(1).strip() if calculo_match else 'N/A'
    }

def processar_pdfs_na_pasta(pasta_path):
    """Função principal que varre uma pasta, processa todos os PDFs e retorna um DataFrame consolidado."""
    arquivos_pdf = [f for f in os.listdir(pasta_path) if f.lower().endswith('.pdf')]
    if not arquivos_pdf:
        print(f"Nenhum arquivo PDF encontrado na pasta: {pasta_path}")
        return None

    lista_geral_funcionarios = []
    print(f"Encontrados {len(arquivos_pdf)} PDFs para processar...")

    for nome_arquivo in arquivos_pdf:
        print(f"\n---> Processando arquivo: {nome_arquivo}")
        try:
            with pdfplumber.open(os.path.join(pasta_path, nome_arquivo)) as pdf:
                texto_completo_pdf = "".join([(page.extract_text(x_tolerance=1, y_tolerance=1) or "") + "\n" for page in pdf.pages])
                info_base = extrair_info_base(texto_completo_pdf)
                depto_map = {match.start(): match.group(1).strip() for match in re.finditer(r'Departamento:\s*(.+)', texto_completo_pdf)}
                depto_indices = sorted(depto_map.keys())
                blocos_encontrados = re.finditer(r'((?:Empr|Contr)\.?\s*:\s*\d+.*?)(?=\n(?:Empr|Contr)\.?\s*:\s*\d+|Resumo por Rubricas|Totais por Departamento)', texto_completo_pdf, re.DOTALL)

                funcionarios_no_arquivo = 0
                for bloco_match in blocos_encontrados:
                    bloco = bloco_match.group(1)
                    if not ("Situação:" in bloco and "CPF:" in bloco): continue

                    posicao_bloco = bloco_match.start()
                    departamento_atual = next((depto_map[idx] for idx in reversed(depto_indices) if idx < posicao_bloco), "N/A")
                    dados_funcionario = {'departamento': departamento_atual, **info_base}

                    # --- INÍCIO DA LÓGICA DE EXTRAÇÃO ROBUSTA ---
                    # Extrai cada campo individualmente para evitar falha total.

                    # Vínculo
                    vinculo_match = re.search(r'(Empr|Contr)\.?', bloco)
                    dados_funcionario['vinculo'] = 'Empregado' if vinculo_match and 'Empr' in vinculo_match.group(0) else 'Contribuinte' if vinculo_match else 'N/A'

                    # Nome (mais flexível)
                    nome_match = re.search(r'(?:Empr|Contr)\.?\s*:\s*\d+\s+(.*?)\s*Situação:', bloco, re.DOTALL)
                    dados_funcionario['nome_funcionario'] = nome_match.group(1).replace('\n', ' ').strip() if nome_match else 'N/A'

                    # CPF
                    cpf_match = re.search(r'CPF:\s*([\d\.\-]+)', bloco)
                    dados_funcionario['cpf'] = cpf_match.group(1).strip() if cpf_match else 'N/A'

                    # Data de Admissão
                    admissao_match = re.search(r'Adm:\s*(\d{2}/\d{2}/\d{4})', bloco)
                    dados_funcionario['data_admissao'] = admissao_match.group(1).strip() if admissao_match else 'N/A'

                    # Cargo
                    cargo_match = re.search(r'Cargo:\s*\d+\s+(.*?)(?=\s+Salário:|\s+C\.|С\.)', bloco, re.DOTALL)
                    dados_funcionario['cargo'] = cargo_match.group(1).replace('\n', ' ').strip() if cargo_match else 'N/A'
                    # --- FIM DA LÓGICA DE EXTRAÇÃO ROBUSTA ---

                    salario_match = re.search(r'Salário:\s*([\d\.,]+)', bloco)
                    if salario_match: dados_funcionario['salario_contratual'] = limpar_valor(salario_match.group(1))

                    rodape_bloco = bloco[bloco.find("ND:"):] if "ND:" in bloco else ""
                    rodape_match = re.search(r'Proventos:\s*([\d\.,]+)\s+Descontos:\s*([\d\.,]+).*?L[íi]quido:\s*([\d\.,]+).*?Base INSS:\s*([\d\.,]+).*?Base FGTS:\s*([\d\.,]+).*?Valor FGTS:\s*([\d\.,]+).*?Base IRRF:\s*([\d\.,]+)', rodape_bloco, re.DOTALL)
                    if rodape_match:
                        dados_funcionario.update({
                            'total_proventos': limpar_valor(rodape_match.group(1)), 'total_descontos': limpar_valor(rodape_match.group(2)),
                            'valor_liquido': limpar_valor(rodape_match.group(3)), 'base_inss': limpar_valor(rodape_match.group(4)),
                            'base_fgts': limpar_valor(rodape_match.group(5)), 'valor_fgts': limpar_valor(rodape_match.group(6)),
                            'base_irrf': limpar_valor(rodape_match.group(7))
                        })

                    inicio_tabela = max(bloco.find("C.B.O:"), bloco.find("С.В.О:"))
                    fim_tabela = bloco.find("\nND:")
                    if inicio_tabela != -1 and fim_tabela != -1:
                        tabela_str = bloco[inicio_tabela:fim_tabela].split('\n')[1:]

                        for linha in tabela_str:
                            if not re.search(r'\d', linha):
                                continue

                            padrao_verba = r'(\d+)\s+(.*?)\s+([\d\.,]+)\s+([PD])(?=\s+\d{2,}|$)'

                            matches = re.finditer(padrao_verba, linha)
                            for match in matches:
                                codigo = match.group(1)
                                descricao_bruta = match.group(2).strip()
                                valor = match.group(3)
                                descricao_bruta = re.sub(r'\s[\d\.,%]+$', '', descricao_bruta).strip()
                                nome_col = limpar_nome_coluna(codigo, descricao_bruta)
                                valor_limpo = limpar_valor(valor)
                                dados_funcionario[nome_col] = dados_funcionario.get(nome_col, 0) + valor_limpo

                    lista_geral_funcionarios.append(dados_funcionario)
                    funcionarios_no_arquivo += 1

                print(f"    - Sucesso! Foram processados {funcionarios_no_arquivo} funcionários neste arquivo.")

        except Exception as e:
            print(f"  ERRO CRÍTICO ao processar o arquivo {nome_arquivo}: {e}")

    if not lista_geral_funcionarios:
        print("\nProcesso concluído, mas nenhum dado de funcionário pôde ser extraído.")
        return None

    df = pd.DataFrame(lista_geral_funcionarios).fillna(0)

    colunas_info_pessoal = [
        'competencia', 'tipo_calculo', 'departamento', 'vinculo', 'nome_funcionario', 'cargo', 'data_admissao', 'cpf',
        'salario_contratual', 'total_proventos', 'total_descontos', 'valor_liquido', 'base_inss', 'base_fgts',
        'valor_fgts', 'base_irrf'
    ]
    colunas_presentes = [col for col in colunas_info_pessoal if col in df.columns]
    colunas_rubricas = sorted([col for col in df.columns if col not in colunas_presentes])

    df = df[colunas_presentes + colunas_rubricas]
    return df

# --- PONTO DE EXECUÇÃO ---
if __name__ == "__main__":
    caminho_da_pasta = '/content/FOPAG'
    df_consolidado = processar_pdfs_na_pasta(caminho_da_pasta)

    if df_consolidado is not None and not df_consolidado.empty:
        nome_arquivo_saida = 'BASE_FOPAAG_STAGGIN.csv'
        df_consolidado.to_csv(nome_arquivo_saida, index=False, sep=';', decimal=',', encoding='utf-8-sig')
        print("\n\n--- Processo Finalizado com Sucesso! ---")
        print(f"Sua base de dados final foi salva no arquivo: {os.path.abspath(nome_arquivo_saida)}")
    else:
        print("\nNenhum dado foi gerado. Verifique se os PDFs estão na pasta correta e não estão corrompidos.")

Encontrados 36 PDFs para processar...

---> Processando arquivo: 01.2023 ARQDIGITAL - Folha de Pagamento c.Prolabore Carol.pdf
    - Sucesso! Foram processados 39 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Folha- 08-2023.pdf
    - Sucesso! Foram processados 47 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Mensal-03-2024.pdf
    - Sucesso! Foram processados 66 funcionários neste arquivo.

---> Processando arquivo: ARQ - 1ª Parcela 13.2023.pdf
    - Sucesso! Foram processados 42 funcionários neste arquivo.

---> Processando arquivo: 05.2023 ARQDIGITAL - Folha de Pagamento.pdf
    - Sucesso! Foram processados 40 funcionários neste arquivo.

---> Processando arquivo: 04.2023 ARQDIGITAL - Folha de Pagamento.pdf
    - Sucesso! Foram processados 40 funcionários neste arquivo.

---> Processando arquivo: 10-Extrato Folha- 01-2024.pdf
    - Sucesso! Foram processados 62 funcionários neste arquivo.

---> Processando arquivo: 12.2024 ARQDIGITAL - Folha de